In [ ]:
import networkx as nx
import xgi
import numpy as np
import networkx as nx

from ising import *
from higher_order_contagion import *
from ho_info_metrics.metrics import dtc_boot, tc_boot

In [ ]:
N = 200
ps = [0.01, 0.0005]
tcs_matrix = np.zeros((12, 10))
dtcs_matrix = np.zeros((12, 10))
for i in range(12):
    print(i)
    S = xgi.random_simplicial_complex(N, ps)
    edges = [list(i) for i in S.edges.filterby("order", 1).members()]
    triangles = [list(i) for i in S.edges.filterby("order", 2).members()]

    triplets = np.array(triangles)
    random_triplets = np.array(
        [random.sample(range(N), 3) for _ in range(len(triplets))]
    )
    triplets = np.vstack((triplets, random_triplets))
    print(triplets.shape)
    N = len(S.nodes)
    J1 = 0.2
    J2s = np.linspace(0.1, 10, 10)
    tcs = []
    dtcs = []

    for i, J2 in enumerate(J2s):
        ts, _, _ = ising_dynamic_parallel(N, edges, triangles, J1, J2, 3000, T=6)
        ts = ts[:, np.random.permutation(ts.shape[1])]
        tc = np.average(
            [
                tc_boot(
                    np.vstack(
                        (ts[:, triplet[0]], ts[:, triplet[1]], ts[:, triplet[2]])
                    ),
                    estimator="cat_ent",
                )
                for triplet in triplets
            ]
        )
        tcs.append(tc)
        dtc = np.average(
            [
                dtc_boot(
                    np.vstack(
                        (ts[:, triplet[0]], ts[:, triplet[1]], ts[:, triplet[2]])
                    ),
                    estimator="cat_ent",
                )
                for triplet in triplets
            ]
        )
        dtcs.append(dtc)
    tcs_matrix[i] = tcs
    dtcs_matrix[i] = dtcs
tcs = np.mean(tcs_matrix, axis=0)
dtcs = np.mean(dtcs_matrix, axis=0)

np.save("./ising_results/tcs.npy", tcs)
np.save("./ising_results/dtcs.npy", dtcs)

In [ ]:
tcs_matrix = np.zeros((12, 10))
dtcs_matrix = np.zeros((12, 10))
for _ in range(12):
    print(_)
    tcs = []
    dtcs = []

    # Simplicial Complex parameters
    N = 300  # number of nodes
    k1_init = 20  # average degree from wich we construct the network
    k2_init = 7  # average hyper-degree (mean number of triangles per node)
    p1, p2 = get_p1_and_p2_correction(k1_init, k2_init, N)

    l1 = 2.5
    l2s = np.linspace(0.1, 10, 10)

    G, node_neighbors_dict, triangles_list = generate_my_simplicial_complex_d2(
        N, p1, p2
    )
    edges = G.edges()
    S = xgi.SimplicialComplex()
    S.add_simplices_from(edges)
    S.add_simplices_from(triangles_list)
    print(S)

    edges = [list(i) for i in S.edges.filterby("order", 1).members()]
    triangles = [list(i) for i in S.edges.filterby("order", 2).members()]

    triplets = np.array(triangles)
    random_triplets = np.array(
        [random.sample(range(N), 3) for _ in range(len(triplets))]
    )
    triplets = np.vstack((triplets, random_triplets))
    print(triplets.shape)

    for l2 in l2s:
        print(">>> Simulating with l2 = {}".format(l2))
        # Real average degree and hyper-degree of the simplicial complex
        k1 = (
            1.0
            * sum([len(v) for v in node_neighbors_dict.values()])
            / len(node_neighbors_dict)
        )
        k2 = 3.0 * len(triangles_list) / len(node_neighbors_dict)

        I0_percentage = 30.0  # percentage of initial infected nodes

        mySimplagionModel = SimplagionModel(
            node_neighbors_dict, triangles_list, I0_percentage
        )

        initial_infected = mySimplagionModel.initial_setup(print_status=False)
        # create and save the initial infected nodes list

        t_max = 3000  # number of time steps
        mu = 0.8  # recovery rate

        beta1 = l1 * mu / k1  # simple-infection rate
        beta2 = l2 * mu / k2  # simplicial-infection rate
        mySimplagionModel.initial_setup(fixed_nodes_to_infect=initial_infected)
        ts = mySimplagionModel.run(t_max, beta1, beta2, mu, print_status=False)

        print("Computing TC")
        tc = np.average(
            [
                tc_boot(
                    np.vstack(
                        (ts[:, triplet[0]], ts[:, triplet[1]], ts[:, triplet[2]])
                    ),
                    estimator="cat_ent",
                )
                for triplet in triplets
            ]
        )
        tcs.append(tc)
        print("Computing DTC")
        dtc = np.average(
            [
                dtc_boot(
                    np.vstack(
                        (ts[:, triplet[0]], ts[:, triplet[1]], ts[:, triplet[2]])
                    ),
                    estimator="cat_ent",
                )
                for triplet in triplets
            ]
        )
        dtcs.append(dtc)

    tcs_matrix[_] = tcs
    dtcs_matrix[_] = dtcs

tcs = np.average(tcs_matrix, axis=0)
dtcs = np.average(dtcs_matrix, axis=0)

In [ ]:
np.save("./simplagion_results/tcs.npy", tcs)
np.save("./simplagion_results/dtcs.npy", dtcs)